**Descrição**

Este script serve como complemento à análise do One-way anova que utilizando análise de variância avaliou estatisticamente a capacidade de cada marca marcador de distinguir individualmente entre as classificações prévias dos grupos (Controle vs Diabetes Mellitus).

Assim, esse script validara parcialmente o desempenho de cada um desses marcadores na classificação dos grupos e ranqueará seus desempenhos baseados nas métricas padrão de Acurácia, Precisão, Sensibilidade (Recall) e F1-score.

**Importação de bibliotecas**

In [23]:
# Limpando ambiente
%reset

# Ignorando mensagens de aviso
import warnings
warnings.filterwarnings('ignore')

#Bibliotecas Relevantes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Biblioteca auxiliar para codificação das Labels
from sklearn.preprocessing import StandardScaler 

# Biblioteca para Cross Validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

# Modelos de Treinamento
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Suporte de métricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

**Métodos para Algoritmos de Classificação**

*SVM com Cross-validation*

In [24]:
def svm_classification_cv(X_treino, y_treino):
    
    # X = dataset = feature vector
    # y = labels = target variable
    
    # Criação do modelo
    svm_model = SVC(kernel='rbf') # Por que esse kernel? Pois não sabemos temos conhecimento prévio padronizado para todas os marcadores.
    
    # Execução do modelo, via Cross-fold x5.
    resultados = cross_validate(svm_model, 
                                X=X_treino, 
                                y=y_treino, 
                                cv=5,
                                scoring = ['accuracy', 'precision', 'recall', 'f1'],
                                return_train_score=True)
    
    # plt.plot(resultados['test_accuracy'], color="red", label='Test')
    # plt.plot(resultados['train_accuracy'], color="blue", label='Train')
    # plt.show()
    
    return [
        resultados['test_accuracy'].mean(),
        resultados['test_precision'].mean(),
        resultados['test_recall'].mean(), 
        resultados['test_f1'].mean(),
        resultados['train_accuracy'].mean(),
        resultados['train_precision'].mean(),
        resultados['train_recall'].mean(), 
        resultados['train_f1'].mean()
    ]

*Árvore de Decisão com Cross-validation*

In [25]:
def arvore_decisao_cv(X_treino, y_treino):
    
    # X = dataset = feature vector
    # y = labels = target variable
    
    # Criação do modelo
    Arvore_decisao_model = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=31415) # Por que esse critério? Pois não sabemos temos conhecimento prévio padronizado para todas os marcadores.
    
    # Execução do modelo, via Cross-fold x5.
    resultados = cross_validate(Arvore_decisao_model, 
                                X=X_treino, 
                                y=y_treino, 
                                cv=5,
                                scoring = ['accuracy', 'precision', 'recall', 'f1'],
                                return_train_score=True)
    
    return [
        resultados['test_accuracy'].mean(),
        resultados['test_precision'].mean(),
        resultados['test_recall'].mean(), 
        resultados['test_f1'].mean(),
        resultados['train_accuracy'].mean(),
        resultados['train_precision'].mean(),
        resultados['train_recall'].mean(), 
        resultados['train_f1'].mean()
    ]

*KNN com Cross-validation*

In [26]:
def knn_classification_cv(X_treino, y_treino):
    
    # X = dataset = feature vector
    # y = labels = target variable
    
    # Criação do modelo
    knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')
    
    # Execução do modelo, via Cross-fold x5.
    resultados = cross_validate(knn_model, 
                                X=X_treino, 
                                y=y_treino, 
                                cv=5,
                                scoring = ['accuracy', 'precision', 'recall', 'f1'],
                                return_train_score=True)
    
    return [
        resultados['test_accuracy'].mean(),
        resultados['test_precision'].mean(),
        resultados['test_recall'].mean(), 
        resultados['test_f1'].mean(),
        resultados['train_accuracy'].mean(),
        resultados['train_precision'].mean(),
        resultados['train_recall'].mean(), 
        resultados['train_f1'].mean()
    ]

Regressão Logística com Cross-validation

In [27]:
def rl_classification_cv(X_treino, y_treino):
    
    # X = dataset = feature vector
    # y = labels = target variable
    
    # Criação do modelo
    rl_model = LogisticRegression(solver='liblinear')
    
    # Execução do modelo, via Cross-fold x5.
    resultados = cross_validate(rl_model, 
                                X=X_treino, 
                                y=y_treino, 
                                cv=5,
                                scoring = ['accuracy', 'precision', 'recall', 'f1'],
                                return_train_score=True)

    return [
        resultados['test_accuracy'].mean(),
        resultados['test_precision'].mean(),
        resultados['test_recall'].mean(), 
        resultados['test_f1'].mean(),
        resultados['train_accuracy'].mean(),
        resultados['train_precision'].mean(),
        resultados['train_recall'].mean(), 
        resultados['train_f1'].mean()
    ]

**Carregamento e preparação de dados**

In [28]:
# Arquivos com marcadores separados por janelas e por classificação prévia (Controle vs Diabetes Mellitus)

dados_sem_outliers_geral_j1 = pd.read_csv(r'C:\Users\AlexA\Meu Drive\UnB - Mestrado - Regular\Projeto e Pesquisa\Repositório de Dados\Dados Pré-processados\CDED-1.0.1-final\Planilhas e Dados de Controle\ML_Dataset\Features_Without_Outliers_GERAL_Pearson_J1.csv')

dados_sem_outliers_geral_j2 = pd.read_csv(r'C:\Users\AlexA\Meu Drive\UnB - Mestrado - Regular\Projeto e Pesquisa\Repositório de Dados\Dados Pré-processados\CDED-1.0.1-final\Planilhas e Dados de Controle\ML_Dataset\Features_Without_Outliers_GERAL_Pearson_J2.csv')

# Arquivos com marcadores separados (com_outliers) por janelas e por classificação prévia (Controle vs Diabetes Mellitus)

dados_com_outliers_geral_j1 = pd.read_csv(r'C:\Users\AlexA\Meu Drive\UnB - Mestrado - Regular\Projeto e Pesquisa\Repositório de Dados\Dados Pré-processados\CDED-1.0.1-final\Planilhas e Dados de Controle\ML_Dataset\Features_With_Outliers_GERAL_Spearman_J1.csv')

dados_com_outliers_geral_j2 = pd.read_csv(r'C:\Users\AlexA\Meu Drive\UnB - Mestrado - Regular\Projeto e Pesquisa\Repositório de Dados\Dados Pré-processados\CDED-1.0.1-final\Planilhas e Dados de Controle\ML_Dataset\Features_With_Outliers_GERAL_Spearman_J2.csv')

# Caminho para salvar matrizes de difusão dos algoritmos

caminho_salvamentos = r'C:\Users\AlexA\Meu Drive\UnB - Mestrado - Regular\Projeto e Pesquisa\Repositório de Dados\Dados Pré-processados\CDED-1.0.1-final\Planilhas e Dados de Controle\ML-1-Feature\Individual'


**Criando tabela para unificação dos Resultados**

In [29]:
# Colunas do dataframe
Colunas_metricas = ['Marcador', 'Algoritmo', 'Metodo', 
                    'Acuracia_Validacao', 'Precisao_Validacao', 'Sensibilidade_Validacao', 'F1Score_Validacao',
                    'Acuracia_Treinamento', 'Precisao_Treinamento', 'Sensibilidade_Treinamento', 'F1Score_Treinamento',
                    'Dados', 'DadosNaoNulos',
                    'DM', 'CONTROLE']

# Dataframes para coletas de métricas
Metricas_KNN = pd.DataFrame(columns = Colunas_metricas)

**Teste de Classificação com SVM sem Cross-fold validation vs com Cross-fold validation**

In [30]:
def SVM_Execucao(dados, nome_arquivo):
    
    # Dataframes para coletas de métricas
    Metricas_SVM = pd.DataFrame(columns = Colunas_metricas)
    
    # Separação entre dados/dataset e Labels.
    dataset = dados.drop(['PATIENT', 'ECG', 'BP', 'ILV', 'TOTAL_AMOSTRAS_RRI', 'LABEL'], axis=1)
    # dataset = dados[['MEDIA', 'MEDIANA']]
    
    dataset.drop(['LF_ILV_WELCH', 'HF_ILV_WELCH', 'LF_HF_RATIO_ILV_WELCH',
                  'ACOPLAM_FIT_ESTIMATION', 'ACOPLAM_FIT_VALIDATION',
                  'ACOPLAM_TIME_TO_PEAK_SAMPLE_SBP', 'ACOPLAM_TIME_TO_PEAK_SAMPLE_ILV',
                  'ABR_FIT_ESTIMATION', 'ABR_FIT_VALIDATION', 
                  'RCC_FIT_ESTIMATION', 'RCC_FIT_VALIDATION'], axis=1, inplace=True)
    
    # Normalizador
    scaler = StandardScaler()
    
    # Iteração sobra colunas/marcadores
    for marcador in dataset:
    
        # Remoção de nulos
        dados_marcador = dados[[marcador, 'LABEL']].copy()
        dados_marcador.dropna(subset=[marcador], inplace=True)
        
        label_marcador = dados_marcador[['LABEL']].copy()
        dados_marcador.drop(['LABEL'], inplace=True, axis=1)
        
        # Normalizando os dados (Mesmo sabendo que diversos marcadores não possuem distribuição normal)
        X_treino_CV = pd.DataFrame(scaler.fit_transform(dados_marcador), columns=[marcador])
        
        # Treinamento e validação dos dados com K-fold Cross-validation
        B = np.array([marcador, 'SVM', 'CV'] + \
            svm_classification_cv(X_treino_CV, label_marcador) + \
            [dados[[marcador]].shape[0]] + \
            [dados_marcador.shape[0]] + \
            [label_marcador.astype(bool).sum(axis=0)[0]] + \
            [dados_marcador.shape[0] - label_marcador.astype(bool).sum(axis=0)[0]]
        )
        
        Metricas_SVM.loc[len(Metricas_SVM)] = B
        
    Metricas_SVM.to_csv(caminho_salvamentos + '\\' + nome_arquivo + '.csv', index=False, na_rep='NaN')


In [31]:
# Chamada do método para execução do algoritmo

SVM_Execucao(dados_sem_outliers_geral_j1.append(dados_sem_outliers_geral_j2, ignore_index=True), r'metricas_SVM_sem_outliers_geral')
SVM_Execucao(dados_com_outliers_geral_j1.append(dados_com_outliers_geral_j2, ignore_index=True), r'metricas_SVM_com_outliers_geral')

**Teste de Classificação com Árvore de Decisão sem Cross-fold validation**

In [32]:
def ARVORE_DECISAO_Execucao(dados, nome_arquivo):
    
    # Dataframes para coletas de métricas
    Metricas_AD = pd.DataFrame(columns = Colunas_metricas)
    
    # Separação entre dados/dataset e Labels.
    dataset = dados.drop(['PATIENT', 'ECG', 'BP', 'ILV', 'TOTAL_AMOSTRAS_RRI', 'LABEL'], axis=1)
    
    dataset.drop(['LF_ILV_WELCH', 'HF_ILV_WELCH', 'LF_HF_RATIO_ILV_WELCH',
                  'ACOPLAM_FIT_ESTIMATION', 'ACOPLAM_FIT_VALIDATION',
                  'ACOPLAM_TIME_TO_PEAK_SAMPLE_SBP', 'ACOPLAM_TIME_TO_PEAK_SAMPLE_ILV',
                  'ABR_FIT_ESTIMATION', 'ABR_FIT_VALIDATION', 
                  'RCC_FIT_ESTIMATION', 'RCC_FIT_VALIDATION'], axis=1, inplace=True)
    
    # Normalizador
    scaler = StandardScaler()
    
    # Iteração sobra colunas/marcadores
    for marcador in dataset:
    
        # Remoção de não nulos
        dados_marcador = dados[[marcador, 'LABEL']].copy()
        dados_marcador.dropna(inplace=True, axis=0)
        
        label_marcador = dados_marcador[['LABEL']].copy()
        dados_marcador.drop(['LABEL'], inplace=True, axis=1)
        
        # Normalizando os dados (Mesmo sabendo que diversos marcadores não possuem distribuição normal)
        X_treino_CV = pd.DataFrame(scaler.fit_transform(dados_marcador), columns=[marcador])

        # Treinamento e validação dos dados com K-fold Cross-validation
        B = np.array([marcador, 'ARVORE_DECISAO', 'CV'] + \
            arvore_decisao_cv(X_treino_CV, label_marcador) + \
            [dados[[marcador]].shape[0]] + \
            [dados_marcador.shape[0]] + \
            [label_marcador.astype(bool).sum(axis=0)[0]] + \
            [dados_marcador.shape[0] - label_marcador.astype(bool).sum(axis=0)[0]]
        )

        Metricas_AD.loc[len(Metricas_AD)] = B

    Metricas_AD.to_csv(caminho_salvamentos + '\\' + nome_arquivo + '.csv', index=False, na_rep='NaN')

In [33]:
ARVORE_DECISAO_Execucao(dados_sem_outliers_geral_j1.append(dados_sem_outliers_geral_j2, ignore_index=True), r'metricas_AD_sem_outliers_geral')
ARVORE_DECISAO_Execucao(dados_com_outliers_geral_j1.append(dados_com_outliers_geral_j2, ignore_index=True), r'metricas_AD_com_outliers_geral')

**Teste de Classificação com K-nearest Neighbour sem Cross-fold validation**

In [34]:
def KNN_Execucao(dados, nome_arquivo):
    
    # Dataframes para coletas de métricas
    Metricas_KNN = pd.DataFrame(columns = Colunas_metricas)
    
    # Separação entre dados/dataset e Labels.
    dataset = dados.drop(['PATIENT', 'ECG', 'BP', 'ILV', 'TOTAL_AMOSTRAS_RRI', 'LABEL'], axis=1)
    
    dataset.drop(['LF_ILV_WELCH', 'HF_ILV_WELCH', 'LF_HF_RATIO_ILV_WELCH',
                  'ACOPLAM_FIT_ESTIMATION', 'ACOPLAM_FIT_VALIDATION',
                  'ACOPLAM_TIME_TO_PEAK_SAMPLE_SBP', 'ACOPLAM_TIME_TO_PEAK_SAMPLE_ILV',
                  'ABR_FIT_ESTIMATION', 'ABR_FIT_VALIDATION', 
                  'RCC_FIT_ESTIMATION', 'RCC_FIT_VALIDATION'], axis=1, inplace=True)
    
    # Normalizador
    scaler = StandardScaler()
    
    # Iteração sobra colunas/marcadores
    for marcador in dataset:
    
        # Remoção de não nulos
        dados_marcador = dados[[marcador, 'LABEL']].copy()
        dados_marcador.dropna(inplace=True, axis=0)

        label_marcador = dados_marcador[['LABEL']].copy()
        dados_marcador.drop(['LABEL'], inplace=True, axis=1)
        
        # Normalizando os dados (Mesmo sabendo que diversos marcadores não possuem distribuição normal)
        X_treino_CV = pd.DataFrame(scaler.fit_transform(dados_marcador), columns=[marcador])

        # Treinamento e validação dos dados com K-fold Cross-validation
        B = np.array([marcador, 'KNN', 'CV'] + \
            knn_classification_cv(X_treino_CV, label_marcador) + \
            [dados[[marcador]].shape[0]] + \
            [dados_marcador.shape[0]] + \
            [label_marcador.astype(bool).sum(axis=0)[0]] + \
            [dados_marcador.shape[0] - label_marcador.astype(bool).sum(axis=0)[0]]
        )

        Metricas_KNN.loc[len(Metricas_KNN)] = B

    Metricas_KNN.to_csv(caminho_salvamentos + '\\' + nome_arquivo + '.csv', index=False, na_rep='NaN')

In [35]:
KNN_Execucao(dados_sem_outliers_geral_j1.append(dados_sem_outliers_geral_j2, ignore_index=True), r'metricas_KNN_sem_outliers_geral')
KNN_Execucao(dados_com_outliers_geral_j1.append(dados_com_outliers_geral_j2, ignore_index=True), r'metricas_KNN_com_outliers_geral')

**Teste de classificação com Regressão Logística**

In [36]:
def RL_Execucao(dados, nome_arquivo):
    
    # Dataframes para coletas de métricas
    Metricas_RL = pd.DataFrame(columns = Colunas_metricas)
    
    # Separação entre dados/dataset e Labels.
    dataset = dados.drop(['PATIENT', 'ECG', 'BP', 'ILV', 'TOTAL_AMOSTRAS_RRI', 'LABEL'], axis=1)
    
    dataset.drop(['LF_ILV_WELCH', 'HF_ILV_WELCH', 'LF_HF_RATIO_ILV_WELCH',
                  'ACOPLAM_FIT_ESTIMATION', 'ACOPLAM_FIT_VALIDATION',
                  'ACOPLAM_TIME_TO_PEAK_SAMPLE_SBP', 'ACOPLAM_TIME_TO_PEAK_SAMPLE_ILV',
                  'ABR_FIT_ESTIMATION', 'ABR_FIT_VALIDATION', 
                  'RCC_FIT_ESTIMATION', 'RCC_FIT_VALIDATION'], axis=1, inplace=True)
    
    # Normalizador
    scaler = StandardScaler()
    
    # Iteração sobra colunas/marcadores
    for marcador in dataset:
    
        # Remoção de não nulos
        dados_marcador = dados[[marcador, 'LABEL']].copy()
        dados_marcador.dropna(inplace=True, axis=0)

        label_marcador = dados_marcador[['LABEL']].copy()
        dados_marcador.drop(['LABEL'], inplace=True, axis=1)
        
        # Normalizando os dados (Mesmo sabendo que diversos marcadores não possuem distribuição normal)
        X_treino_CV = pd.DataFrame(scaler.fit_transform(dados_marcador), columns=[marcador])

        # Treinamento e validação dos dados com K-fold Cross-validation
        B = np.array([marcador, 'RL', 'CV'] + \
            rl_classification_cv(X_treino_CV, label_marcador) + \
            [dados[[marcador]].shape[0]] + \
            [dados_marcador.shape[0]] + \
            [label_marcador.astype(bool).sum(axis=0)[0]] + \
            [dados_marcador.shape[0] - label_marcador.astype(bool).sum(axis=0)[0]]
        )

        Metricas_RL.loc[len(Metricas_RL)] = B

    Metricas_RL.to_csv(caminho_salvamentos + '\\' + nome_arquivo + '.csv', index=False, na_rep='NaN')

In [37]:
RL_Execucao(dados_sem_outliers_geral_j1.append(dados_sem_outliers_geral_j2, ignore_index=True), r'metricas_RL_sem_outliers_geral')
RL_Execucao(dados_com_outliers_geral_j1.append(dados_com_outliers_geral_j2, ignore_index=True), r'metricas_RL_com_outliers_geral')